In [1]:
import torch
import torch.nn as nn
import networkx as nx
import matplotlib.pyplot as plt
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork, DynapcnnNetworkGraph
import sinabs as snb

In [2]:
torch.manual_seed(0)

## Dummy Initialization

In [3]:
channels = 1
height = 28
width = 28

input_shape = (channels, height, width)

In [4]:
ann = nn.Sequential(
    nn.Conv2d(1, 20, 5, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2,2),
    nn.Conv2d(20, 32, 5, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2,2),
    nn.Conv2d(32, 128, 3, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2,2),
    nn.Flatten(),
    nn.Linear(128, 500, bias=False),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(500, 10, bias=False),
)

In [5]:
sinabs_model = from_model(ann, add_spiking_output=True, batch_size=1)

In [6]:
hw_model = DynapcnnNetworkGraph(
    sinabs_model,
    discretize=True,
    input_shape=input_shape
)


 [ ENTERED build_from_graph() ]

node to DynapcnnLayer mapping:
DynapcnnLayer index:  0
     [0 <class 'torch.nn.modules.conv.Conv2d'>]
     [1 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [2 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  1
     [3 <class 'torch.nn.modules.conv.Conv2d'>]
     [4 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [5 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  2
     [6 <class 'torch.nn.modules.conv.Conv2d'>]
     [7 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [8 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  3
     [9 <class 'torch.nn.modules.linear.Linear'>]
     [10 <class 'sinabs.layers.iaf.IAFSqueeze'>]
DynapcnnLayer index:  4
     [11 <class 'torch.nn.modules.linear.Linear'>]
     [12 <class 'sinabs.layers.iaf.IAFSqueeze'>]

DynapcnnLayer to DynapcnnLayer mapping:
DynapcnnLayer 0 destinations: [1]
DynapcnnLayer 1 destinations: [2]
DynapcnnLayer 2 destinations: [3]
DynapcnnLayer 3 destina

## Valide Non-sequential Edges

### Test 1

In [7]:
dummy_layer_list = [
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),]

In [8]:
dummy_edges = [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6),
    (6, 7),
    (7, 8),
    (8, 9),
    (9, 10),
    (10, 11),
    (10, 13),
    (11, 12),
    (12, 13),
    (13, 14),]

In [9]:
build_from_graph = DynapcnnNetworkGraph.build_from_graph_()

In [10]:
build_from_graph(dummy_layer_list, None, dummy_edges)


 [ ENTERED build_from_graph() ]

node to DynapcnnLayer mapping:
DynapcnnLayer index:  0
     [0 <class 'torch.nn.modules.conv.Conv2d'>]
     [1 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [2 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  1
     [3 <class 'torch.nn.modules.conv.Conv2d'>]
     [4 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [5 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  2
     [6 <class 'torch.nn.modules.conv.Conv2d'>]
     [7 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [8 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  3
     [9 <class 'torch.nn.modules.linear.Linear'>]
     [10 <class 'sinabs.layers.iaf.IAFSqueeze'>]
DynapcnnLayer index:  4
     [13 <class 'torch.nn.modules.linear.Linear'>]
     [14 <class 'sinabs.layers.iaf.IAFSqueeze'>]
DynapcnnLayer index:  5
     [11 <class 'torch.nn.modules.linear.Linear'>]
     [12 <class 'sinabs.layers.iaf.IAFSqueeze'>]

DynapcnnLayer to DynapcnnLayer mapping:
D

(None, None)

### Test 2

In [11]:
dummy_layer_list = [
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),
    torch.nn.Linear(in_features=128, out_features=500, bias=False),
    snb.layers.iaf.IAFSqueeze(batch_size=1),]

In [12]:
dummy_edges = [
    (0, 1),
    (1, 2),
    (1, 6),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6),
    (6, 7),
    (7, 8),
    (8, 9),
    (9, 10),
    (10, 11),
    (10, 13),
    (11, 12),
    (12, 13),
    (13, 14),]

In [13]:
build_from_graph = DynapcnnNetworkGraph.build_from_graph_()

In [14]:
build_from_graph(dummy_layer_list, None, dummy_edges)


 [ ENTERED build_from_graph() ]

node to DynapcnnLayer mapping:
DynapcnnLayer index:  0
     [0 <class 'torch.nn.modules.conv.Conv2d'>]
     [1 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [2 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  1
     [3 <class 'torch.nn.modules.conv.Conv2d'>]
     [4 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [5 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  2
     [6 <class 'torch.nn.modules.conv.Conv2d'>]
     [7 <class 'sinabs.layers.iaf.IAFSqueeze'>]
     [8 <class 'torch.nn.modules.pooling.AvgPool2d'>]
DynapcnnLayer index:  3
     [9 <class 'torch.nn.modules.linear.Linear'>]
     [10 <class 'sinabs.layers.iaf.IAFSqueeze'>]
DynapcnnLayer index:  4
     [13 <class 'torch.nn.modules.linear.Linear'>]
     [14 <class 'sinabs.layers.iaf.IAFSqueeze'>]
DynapcnnLayer index:  5
     [11 <class 'torch.nn.modules.linear.Linear'>]
     [12 <class 'sinabs.layers.iaf.IAFSqueeze'>]

DynapcnnLayer to DynapcnnLayer mapping:
D

(None, None)